In [1]:
#Importing packages
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from dotenv import load_dotenv
import psycopg2

In [2]:
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

In [3]:
try:
    conn = psycopg2.connect(DATABASE_URL)
    print("Connected to the database")
    cursor = conn.cursor()
    sql = """ SELECT id,name FROM \"Tags\" """
    cursor.execute(sql)
    db_tags = cursor.fetchall()
    dict_tags = {}
    for i in db_tags:
        dict_tags[i[0]] = i[1]
        
    feature_list =["uuid"]+[i[0] for i in db_tags]
    df = pd.DataFrame(columns=feature_list)
    sql = """  SELECT \"poemId\",\"tagId\" FROM  \"PoemsTags\" """
    cursor.execute(sql)
    poems_tags = cursor.fetchall()
   
    dict_poems_tags = {}
    for i in poems_tags:
        if i[0] in dict_poems_tags:
            dict_poems_tags[i[0]][i[1]] = 1
        else:
            dict_poems_tags[i[0]] = {}
            dict_poems_tags[i[0]]['uuid'] = i[0]
            dict_poems_tags[i[0]][i[1]] = 1
    
    for i in dict_poems_tags:
        df.loc[len(df)] =   dict_poems_tags[i]
        

    df = df.fillna(0)
    #df.to_csv('model.csv',index=False)
except Exception as e:
    print("Database Error",e)
finally:
    conn.close()

Connected to the database


In [5]:
data =  df.drop(['uuid'],axis=1)
neigh = NearestNeighbors(n_neighbors=7, metric='cosine',algorithm='brute')
neigh.fit(data)

try:
    conn = psycopg2.connect(DATABASE_URL)
    print("Connected to the database")
    cursor = conn.cursor()
    
    for i in dict_poems_tags:
        post_data = {}
        for tag in dict_tags:
            if tag in dict_poems_tags[i]:
                post_data[tag] = 1
            else:
                post_data[tag] = 0
        post = pd.DataFrame(data=post_data,index=[0])
    
        distances,indices = neigh.kneighbors(post)
        
        for idx in range(len(distances.flatten())):
            #  print('{0}: {1}, with a distance of {2}.'.format(i+1, movies['Movie Name'].iloc[indices.flatten()[i]],distances.flatten()[i]))
            uuid = (df['uuid'].iloc[indices.flatten()[idx]])
            if uuid!=dict_poems_tags[i]['uuid']:
                sql = """ INSERT INTO \"Recommendations\" (\"postId\",\"postRecommendId\",\"score\") VALUES (%s,%s,%s) """
                values = (dict_poems_tags[i]['uuid'],uuid,float(distances.flatten()[idx]),)
                cursor.execute(sql,values)
                conn.commit()
                print(values)
except Exception as e:
    print("Error",e)
finally:
    conn.close()
    

Connected to the database
('24ad22a8-37c0-4623-9fe9-b968d35d9904', '2b15cbb4-4380-4421-a876-0bd5dfabda78', 0.5)
('24ad22a8-37c0-4623-9fe9-b968d35d9904', 'c9e71a4a-c8f4-4f23-b8ed-f2dec0a67cfb', 0.5)
('24ad22a8-37c0-4623-9fe9-b968d35d9904', '62d8206d-c624-4315-a463-573792dc426d', 0.5)
('24ad22a8-37c0-4623-9fe9-b968d35d9904', 'f0211739-60f7-4cbb-b99b-9e01e396fe24', 0.5)
('24ad22a8-37c0-4623-9fe9-b968d35d9904', '96d402b8-4734-4191-90c5-9fb669a08516', 0.5)
('24ad22a8-37c0-4623-9fe9-b968d35d9904', 'f4c826b2-4158-40c2-bf18-97e511265015', 0.5)
('51de802d-2f5f-4d7c-a7c9-343bfd7a2e0d', 'c6ee985c-4fe3-4e9d-abc9-c22a290322fe', 0.5)
('51de802d-2f5f-4d7c-a7c9-343bfd7a2e0d', 'bda05396-0283-443d-b275-0fb786b2c2b6', 0.75)
('51de802d-2f5f-4d7c-a7c9-343bfd7a2e0d', '21f83f6f-2bf5-4e11-808d-b01ef2fe68d4', 0.75)
('51de802d-2f5f-4d7c-a7c9-343bfd7a2e0d', '4cc959be-c9a4-4113-a4eb-515467dcdba8', 0.75)
('51de802d-2f5f-4d7c-a7c9-343bfd7a2e0d', '52ad65a0-e563-4098-8773-44efbea052d0', 0.75)
('51de802d-2f5f-4d7c-a7c